# What words and phrases distinguish two groups of documents?

Author: Daniel Low
Using the scattertext package


Paper: Kessler, J. (2017, July). Scattertext: a Browser-Based Tool for Visualizing how Corpora Differ. In Proceedings of ACL 2017, System Demonstrations (pp. 85-90).

Tutorials:
- https://github.com/JasonKessler/scattertext
- https://colab.research.google.com/drive/1snxAP8X6EIDi42FugJ_h5U-fBGDCqtyS#scrollTo=Zmo4wLW36Xuq 

Parameter descriptions are not well described, but review tutorials and
- https://github.com/JasonKessler/scattertext/blob/master/scattertext/__init__.py
- The "compaction" process allows users to eliminate terms which may not be associated with a category using a variety of feature selection methods.  The issue with this is that the terms eliminated during the selection process are not taken into account when scaling term positions.


In [ ]:
# !pip install matplotlib==3.6.0 spacy==3.6.1 scattertext==0.1.19 pytextrank==3.2.5 textalloc==0.0.3
# !python -m spacy download en_core_web_sm

In [ ]:
import matplotlib
matplotlib.__version__

In [ ]:
# !pip install -q pytextrank==3.2.5
import pytextrank, spacy
import scattertext as st


In [ ]:
from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas
import scattertext as st

on_google_drive = False

if on_google_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    input_dir = '/content/drive/MyDrive/datum/rallypoint_suicide_detection/data/input/'
    output_dir = '/content/drive/MyDrive/datum/rallypoint_suicide_detection/data/output/'
else:
    input_dir = './data/input/final_datasets/'
    output_dir = './data/output/semantic_analysis/'
    

In [ ]:
train = pd.read_csv(input_dir + 'train.csv')


In [ ]:
text_col_name = 'content'
group_col_name = 'Group'
category = 'suicidal'
not_category = 'nonsuicidal'
minimum_term_frequency=7
compaction = 2000 

In [ ]:
train[group_col_name] = [category if n==1 else not_category for n in train['label'].values]

In [ ]:

class LogOddsRatioSmoothedZScorePrior:
    def __init__(self, prior, prior_scale):
        self.prior = prior
        self.prior_scale = prior_scale
    def get_scores(self, a, b): 
        ap = a + self.prior * self.prior_scale*sum(a)/sum(self.prior.values)
        bp = b + self.prior * self.prior_scale*sum(b)/sum(self.prior.values)
        lor = (np.log(ap/(np.sum(ap) - ap)) - np.log(bp/(np.sum(bp) - bp)))
        lorstd = 1./ap + 1./(np.sum(ap) - ap) + 1./bp + 1./(np.sum(bp) - bp)
        return lor/np.sqrt(lorstd)
        
    def get_name(self): 
        return 'Log-Odds-Ratio w/ Informative Dirichlet Prior Z-Score'





def return_method(method_name, priors = None, odds_ratio_prior_n = 10):

    if method_name == 'fscore':
        return st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf')
    elif method_name == 'rank_difference':
        return RankDifference()
    # elif method_name == 'dense_rank':
    #     return st.Scalers.dense_rank
    elif method_name == 'odds_ratio':
        return st.LogOddsRatioInformativeDirichletPrior(priors, odds_ratio_prior_n, 'class-size')


# Preprocess

In [ ]:
%%time



# Unigram corpus 
# =======================================
df_parsed = train.assign(
    parse=lambda df: df[text_col_name].apply(st.whitespace_nlp_with_sentences)
)


corpus_unigram = st.CorpusFromParsedDocuments(
    df_parsed, category_col=group_col_name, parsed_col='parse'
).build().get_unigram_corpus().compact(st.AssociationCompactor(compaction))




def return_corpus(ngram):
    if ngram == 'word':
        return corpus_unigram
    
    elif ngram == 'phrases':
        return corpus_phrases
    



In [ ]:
%%time 
compaction_phrases = 4000

# Using Phrase Machine phrase detector
# =======================================
corpus_phrases = (CorpusFromPandas(train,
                           category_col=group_col_name,
                           text_col=text_col_name,
                           feats_from_spacy_doc=PhraseMachinePhrases(),
                           # nlp=spacy.load('en', parser=False)
                              nlp = spacy.load("en_core_web_sm")
                             ).build().compact(AssociationCompactor(compaction_phrases)))

# Word scores

## Words scores (axes are term frequency in each group, color-coded by F-scores)

In [ ]:
ngram = 'word' # {'word', 'phrases'}
score_method = 'fscore' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'freq' # {'freq', 'score'}
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'

html = st.produce_scattertext_explorer(
    return_corpus(ngram),
    category=category, category_name=category, not_category_name=not_category,
    minimum_term_frequency=minimum_term_frequency, pmi_threshold_coefficient=0,
    width_in_pixels=1000, 
    transform=st.Scalers.percentile_alphabetical,
    # metadata=corpus.get_df()['type'],
    # transform = default is percentile_alphabetical?, #st.Scalers.scale_center_zero_abs, #{dense_rank, st.Scalers.scale_center_zero_abs(vec)}
                                       # https://github.com/JasonKessler/scattertext/blob/b41e3a875faf6dd886e49e524345202432db1b21/scattertext/Scalers.py#L217
    # scores
    term_scorer=return_method(score_method)
)
open(output_dir+filename+'.html', 'w').write(html)
corpus_unigram_clean = corpus.remove_terms(terms = ['groupforsuicidalactivedutyandforsuicidalveterans'])

# just copy it but add return_scatterplot_structure=True to output the structure
scattertext_structure = produce_scattertext_explorer(
    
    # return_corpus(ngram),
    corpus_unigram_clean,
                                    category=category,
                                    category_name=category,
                                    not_category_name=not_category,
                                    minimum_term_frequency=minimum_term_frequency,
                                    pmi_threshold_coefficient=0,
                                    transform=st.Scalers.percentile_alphabetical,
                                    # metadata=corpus.get_df()['speaker'],
                                    term_scorer=return_method(score_method),
                                    width_in_pixels=1000,
    x_label = 'Log frequency in '+not_category+' posts',
    y_label = 'Log frequency in '+category+' posts',
    
                                 return_scatterplot_structure=True,
                                                    )



fig = st.produce_scattertext_pyplot(scattertext_structure)
fig.savefig(output_dir+f'{filename}.tiff', format='tiff', dpi = 150)
fig.savefig(output_dir+f'{filename}.png', format='png', dpi = 150)

In [ ]:
dispersion = st.Dispersion(return_corpus(ngram))
dispersion_df = dispersion.get_df()
display(dispersion_df)

term_category_scores = return_corpus(ngram).get_metadata_freq_df('')
print(term_category_scores)

### Save as figure

In [ ]:
[n for n in corpus.get_terms() if n.startswith('groupfor')]

## Word scores (axes are F-score as a function of term frequency in corpus, color-coded by F-scores)

In [ ]:
ngram = 'word' # {'word', 'phrases'}
score_method = 'fscore' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'score' # {'freq', 'score'}
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'

html = st.produce_fightin_words_explorer(
    corpus_unigram_clean,
    category=category,
    category_name=category,
    not_category_name=not_category,
    not_categories=[not_category],
    neutral_categories=[],
    transform=st.Scalers.percentile_alphabetical,
    minimum_term_frequency=minimum_term_frequency,
    term_scorer=st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),
    # metadata = rdf['movie_name'],
    grey_threshold=0.1
)

open(output_dir+filename+'.html', 'w').write(html)


# just copy it but add return_scatterplot_structure=True to output the structure


In [ ]:

scattertext_structure._y_axis_labels

In [ ]:
dispersion = st.Dispersion(corpus_unigram_clean).get_df()
dispersion['Frequency'].describe()

In [ ]:
# parsed = np.array([n.text for n in corpus_unigram_clean.get_df()['parse'].values])
# parsed
# print(len(parsed), 'documents')

In [ ]:


# tokens = []
# for n in parsed:
#     try: tokens.append(n.replace('\n',' ').replace('\t', ' '.split(' '))
#     except: 
#         print(n)
# tokens = np.unique([n for i in tokens for n in i])
# counts_df = pd.DataFrame(counts, index = ['counts']).T
# counts_df[counts_df['counts']>=2] #none, because I need to parse tokens correctly removing punctuation, lower case
# counts_df


In [ ]:

scattertext_structure = st.produce_fightin_words_explorer(
corpus_unigram_clean,
    category=category,
    category_name=category,
    not_category_name=not_category,
    not_categories=[not_category],
    neutral_categories=[],
    transform=st.Scalers.percentile_alphabetical,
    minimum_term_frequency=minimum_term_frequency,
    term_scorer=st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),
    # metadata = rdf['movie_name'],
    grey_threshold=0.1,

    
    # y_axis_values = [-1,0,1],
    return_scatterplot_structure=True
)
# Change plotting params add '_' at the beginning:
scattertext_structure._x_label = 'Log frequency in entire corpus'
scattertext_structure._y_label = 'Non-suicidal vs. suicidal (Scaled F-Score)'

# Change plotting params: I think it requires 3 values
log_freq_distr = dispersion['Frequency'].values
min_freq = int(np.min(log_freq_distr))
max_freq = int(np.max(log_freq_distr))
scattertext_structure._x_axis_labels=[min_freq, '',max_freq]
# https://github.com/JasonKessler/scattertext/blob/b41e3a875faf6dd886e49e524345202432db1b21/scattertext/viz/PyPlotFromScattertextStructure.py#L149
scattertext_structure._y_axis_labels=[-1,0,1]#[f'{not_category} (-1)','0',f'{category} (1)']



fig = st.produce_scattertext_pyplot(scattertext_structure, textsize = 8, dpi=300)
# plt.xticks(ticks = [-1,0,1],labels=[-1,0,1])
plt.tight_layout()
plt.show()
fig.savefig(output_dir+f'{filename}.tiff', format='tiff', dpi = 150)
fig.savefig(output_dir+f'{filename}.png', format='png', dpi = 150)


### List sentences with top words


In [ ]:
n=100
term_freq_df = corpus_unigram.get_term_freq_df()
term_freq_df[f'{category} score'] = corpus.get_scaled_f_scores(category)
top_words_category = list(term_freq_df.sort_values(by=f'{category} score', ascending=False).index[:n])
top_words_category_score = corpus_unigram.get_scaled_f_scores(category)[:n]
# top_words_not_category = list(term_freq_df.sort_values(by=f'{category} score', ascending=False).index[::-1][:n])
# top_words_not_category_score = corpus_unigram.get_scaled_f_scores(category)[::-1][:n]

df_i = corpus_unigram.get_df()
df_category = df_i[df_i[group_col_name]==category]
df_not_category = df_i[df_i[group_col_name]==not_category]

import re


def return_sample_word_in_docs(df, words, text_col_name = 'content', sample_n = 5):
    words_to_docs = {}
    words_to_docs_windows = {}
    for word in words:
        if word == 'i':
            word = 'I'
        elif word == 'm':
            word = "I'm"
        # elif word  == 
        docs_with_word = df[df[text_col_name].str.contains(word)][text_col_name].sample(n=sample_n)
        assert docs_with_word.shape[0] == sample_n
        docs_with_word = docs_with_word.values
        docs_with_word = [n.replace('\n', ' ') for n in docs_with_word]
        words_to_docs[word] = '\n'.join(docs_with_word)
        
        regex_str = r'(?:\b\S+\s*){,7}\b'+ re.escape(word) +r'\b(?:\s*\S+\s*?){,7}'
        

        # from each post, return n words before and after target word.
        docs_with_word_windows = [re.findall(regex_str, doc) for doc in docs_with_word]
        
        docs_with_word_windows = [n for i in docs_with_word_windows for n in i]
        words_to_docs_windows[word] = '\n'.join(docs_with_word_windows)
    return words_to_docs, words_to_docs_windows
        
words_to_docs_category_d, words_to_docs_windows_category_d  = return_sample_word_in_docs(df_category, top_words_category, sample_n = 5)
# docs_not_category_d = return_sample_word_in_docs(df_not_category, top_words_not_category, sample = 5)


In [ ]:
# docs_with_word_windows = [re.findall(regex_str, doc)[0] for doc in docs_with_word]


In [ ]:
output_dir

In [ ]:
word_examples = pd.DataFrame(words_to_docs_windows_category_d, index = ['Examples']).T
word_examples.to_csv(output_dir+filename+f'_examples_top_{n}.csv')

In [ ]:
# n = 15

# for word in top_words[:n]:
#     print(words_to_docs_windows_category_d.get(word))

In [ ]:
len(scattertext_structure._visualization_data.word_dict["data"])

## Word scores (axes are weighted log odds ratio as a function of term frequency in corpus, color-coded by weighted log odds ratio)


### Log-Odds-Ratio with Informative Dirichlet Prior Z-Score
### Prefered, currently very popular in NLP and CSS literature, flaws exist
### Note: this formulation is modified slightly from Monroe et al. 2008.

We know, intuitively, words which which are used very frequently require much more evidence to be significantly associated with a class than less frequent words.  For example, words like "best" and "entertaining" in the above example should be scored lower than "and" and "of".  

The approach developed by Monroe et. al is to more aggressively smooth terms based on their frequency in a background corpus.

The downside is of this approach is that obtaining a background corpus may be difficult. Lists of background term frequencies, especially for 2+grams, can be very large and unwieldy. Moreover, term frequency lists can become outdated very quickly, especially wrt to politics, pop culture, and technology.

We construct two vectors of background term counts for each category, $\alpha_a$ and $\alpha_b$.  The priors are based on a background set of word frequencies, referred to as $y_{c}$  These are ideally in the same domain as the documents being studied, although could come from a general list of word frequencies.  The background counts are normalized to the size of their respective categories. The hyperparameter $\sigma$ scales the background counts to a multiple of their category size. In other words, as $\sigma$ increases, the regularization increases, and the odds ratio tends more toward zero.

The $\sigma$ I've found most useful is 10, which I include in Scattertext.

$$ \alpha_a = \sigma \cdot n_a \cdot \frac{y_{ci}}{n_c} $$

$$ \alpha_b = \sigma \cdot n_b \cdot \frac{y_{ci}}{n_c} $$


The $\alpha$s can then be used analogously to find the Z-Score of the LOR.

$$ \mbox{LOR}(\mbox{term}_i, \mbox{category}_a, \mbox{category}_b) = \log \frac{y_{ai} + \alpha_a}{n_{a} + \alpha_a \cdot |y| - y_{ai} - \alpha_a} - \log \frac{y_{bi} + \alpha_b}{n_{b} + \alpha_b \cdot |y| - y_{bi} - \alpha_b} $$

$$ \mbox{LOR-SE}(\mbox{term}_i, \mbox{category}_a, \mbox{category}_b) = \frac{1}{y_{ai} + \alpha_a} + \frac{1}{n_{a} + \alpha_a \cdot |y| - y_{ai} - \alpha_a} + \frac{1}{y_{bi} + \alpha_b}+ \frac{1}{n_{b} + \alpha_b \cdot |y| - y_{bi} - \alpha_b} $$

Monroe et al. used a different approach to finding $\alpha$, scaling the sum of the peusdocounts to the mean number of words in a document.

$$ \alpha_a = \alpha_b = \overline{|d_{y}|} \frac{y_{ci}}{\sum n_{c}} $$

Jurafsky et al. suggested using raw background corpus counts as $\alpha$.

$$ \alpha_a = \alpha_b = y_c $$

In the following example, we use the plot description as a background corpus using the first method.

* Dan Jurafsky, Victor Chahuneau, Bryan Routledge, and Noah Smith. Narrative framing of consumer sentiment in online restaurant reviews. First Monday. 2014.

In [ ]:
priors_unigram = (st.PriorFactory(corpus_unigram, 
                  category=category, 
                  not_categories=[not_category],
                  starting_count=0.01)
  .use_neutral_categories()
  .get_priors())

In [ ]:
ngram = 'word' # {'word', 'phrases'}
score_method = 'odds_ratio' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'score' # {'freq', 'score'}
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'

html = st.produce_fightin_words_explorer(
    corpus_unigram,
    category=category,
    category_name=category,
    not_category_name=not_category,
    not_categories=[not_category],
    neutral_categories=[],
    transform=st.Scalers.percentile_alphabetical,
    minimum_term_frequency=minimum_term_frequency,
    term_scorer=st.LogOddsRatioInformativeDirichletPrior(priors_unigram, 10, 'class-size'),   # also try: st.LogOddsRatioInformativeDirichletPrior(priors, 1, 'corpus-size')
    # metadata = rdf['movie_name'],
    grey_threshold=0.1
)

open(output_dir+filename+'.html', 'w').write(html)


# just copy it but add return_scatterplot_structure=True to output the structure


In [ ]:
ngram = 'word' # {'word', 'phrases'}
score_method = 'odds_ratio' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'score' # {'freq', 'score'}
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'


scattertext_structure = st.produce_fightin_words_explorer(
corpus_unigram,
    category=category,
    category_name=category,
    not_category_name=not_category,
    not_categories=[not_category],
    neutral_categories=[],
    transform=st.Scalers.percentile_alphabetical,
    minimum_term_frequency=minimum_term_frequency,
    term_scorer=st.LogOddsRatioInformativeDirichletPrior(priors_unigram, 10, 'class-size'),   # also try: st.LogOddsRatioInformativeDirichletPrior(priors, 1, 'corpus-size')
    # metadata = rdf['movie_name'],
    grey_threshold=0.1,

    
    # y_axis_values = [-1,0,1],
    return_scatterplot_structure=True
)
# Change plotting params add '_' at the beginning:
# scattertext_structure._x_label = 'Log frequency in entire corpus'
# scattertext_structure._y_label = 'Non-suicidal vs. suicidal (Scaled F-Score)'

# Change plotting params: I think it requires 3 values
log_freq_distr = dispersion['Frequency'].values
min_freq = int(np.min(log_freq_distr))
max_freq = int(np.max(log_freq_distr))
scattertext_structure._x_axis_labels=[min_freq, '',max_freq]
# https://github.com/JasonKessler/scattertext/blob/b41e3a875faf6dd886e49e524345202432db1b21/scattertext/viz/PyPlotFromScattertextStructure.py#L149
# scattertext_structure._y_axis_labels=[-1,0,1]#[f'{not_category} (-1)','0',f'{category} (1)']



fig = st.produce_scattertext_pyplot(scattertext_structure, textsize = 8, dpi=300)
# plt.xticks(ticks = [-1,0,1],labels=[-1,0,1])
plt.tight_layout()
plt.show()
fig.savefig(output_dir+f'{filename}.tiff', format='tiff', dpi = 150)
fig.savefig(output_dir+f'{filename}.png', format='png', dpi = 150)


# Phrase scores

Phrasemachine from AbeHandler (Handler et al. 2016) uses regular expressions over sequences of part-of-speech tags to identify noun phrases. This has the advantage over using spaCy's NP-chunking in that it tends to isolote meaningful, large noun phases which are free of appositives.


## Phrase scores (axes are term frequency in each group, color-coded by F-scores)

dense-rank difference: a score which is used in most of the two-category contrastive plots here

maximum category-specific score: most prominent phrases in each category, regardless of the prominence in the other category

Fscore: st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),

In [ ]:
dispersion_phrases = st.Dispersion(corpus_phrases).get_df()
dispersion_phrases

In [ ]:
ngram = 'phrases' # {'word', 'phrases'}
score_method = 'fscore' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'freq' # {'freq', 'score'}

minimum_term_frequency=10
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'


html = produce_scattertext_explorer(return_corpus(ngram),
                                    category=category,
                                    category_name=category,
                                    not_category_name=not_category,
                                    minimum_term_frequency=minimum_term_frequency,
                                    pmi_threshold_coefficient=0,
                                    transform=st.Scalers.percentile_alphabetical,
                                        # grey_threshold=0.1,

                                    # metadata=corpus.get_df()['speaker'],
                                    term_scorer=st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),
                                    width_in_pixels=1000)

open(output_dir+f'{filename}.html', 'w').write(html)


In [ ]:
return_method(score_method)

In [ ]:
ngram = 'phrases' # {'word', 'phrases'}
score_method = 'fscore' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'freq' # {'freq', 'score'}

minimum_term_frequency=10
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'

# same as above with     return_scatterplot_structure=True
scattertext_structure = st.produce_scattertext_explorer(
    corpus_phrases,
    category=category,
    category_name=category,
    not_category_name=not_category,
    not_categories=[not_category],
    neutral_categories=[],
    transform=st.Scalers.percentile_alphabetical,
    # transform=dense_rank,
    minimum_term_frequency=0,
    term_scorer=st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),
    # metadata = rdf['movie_name'],
    # grey_threshold=0,

    
    # y_axis_values = [-1,0,1],
    return_scatterplot_structure=True
)
# Change plotting params add '_' at the beginning:
# scattertext_structure._x_label = 'Log frequency in entire corpus'
# scattertext_structure._y_label = 'Non-suicidal vs. suicidal (Scaled F-Score)'

# Change plotting params: I think it requires 3 values
# log_freq_distr = dispersion['Frequency'].values
# min_freq = int(np.min(log_freq_distr))
# max_freq = int(np.max(log_freq_distr))
# scattertext_structure._x_axis_labels=[min_freq, '',max_freq]
# https://github.com/JasonKessler/scattertext/blob/b41e3a875faf6dd886e49e524345202432db1b21/scattertext/viz/PyPlotFromScattertextStructure.py#L149
# scattertext_structure._y_axis_labels=[-1,0,1]#[f'{not_category} (-1)','0',f'{category} (1)']



fig = st.produce_scattertext_pyplot(scattertext_structure, textsize = 8, dpi=300)
# plt.xticks(ticks = [-1,0,1],labels=[-1,0,1])
plt.tight_layout()
plt.show()
# fig.savefig(output_dir+f'{filename}.tiff', format='tiff', dpi = 150)
# fig.savefig(output_dir+f'{filename}.png', format='png', dpi = 150)




## Phrase scores (axes are F-score as a function of term frequency in corpus, color-coded by F-scores)

In [ ]:
corpus_phrases.

In [ ]:
ngram = 'phrases' # {'word', 'phrases'}
score_method = 'fscore' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'score' # {'freq', 'score'}

minimum_term_frequency=0
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'


html = st.produce_fightin_words_explorer(
    # return_corpus(ngram),
    corpus_phrases,
    filter_unigrams = False,
    category=category,
    category_name=category,
    not_category_name=not_category,
    not_categories=[not_category],
    neutral_categories=[],
    transform=st.Scalers.percentile_alphabetical,
    minimum_term_frequency=minimum_term_frequency,
    term_scorer=st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),
    # metadata = rdf['movie_name'],
    # grey_threshold=0.1
)

open(output_dir+f'{filename}.html', 'w').write(html)




In [ ]:


# html = st.produce_frequency_explorer(
#     corpus.remove_terms(set(corpus.get_terms()) - set(term_freq_df.index)),
#     category=category,
# category_name=category,
# not_category_name=not_category,
#     not_categories=[not_category],
#     term_scorer=st.ScaledFScorePresets(beta=1, one_to_neg_one=True),
#     # metadata=rdf['movie_name'],
#     grey_threshold=0
# )

# open(output_dir+f'{filename}.html', 'w').write(html)


## Phrase scores (axes are weighted log odds ratio as a function of term frequency in corpus, color-coded by weighted log odds ratio)

In [ ]:
priors_phrases = (st.PriorFactory(corpus_phrases, 
                  category=category, 
                  not_categories=[not_category],
                  starting_count=0.01)
  .use_neutral_categories()
  .get_priors())

In [ ]:
ngram = 'phrases' # {'word', 'phrases'}
score_method = 'fscore' # {'fscore','rank_difference', 'dense_rank', 'odds_ratio'}
axes = 'freq' # {'freq', 'score'}

minimum_term_frequency=0
filename = f'{ngram}_method-{score_method}_axes-{axes}_compaction-{compaction}-freq-{minimum_term_frequency}'

html = produce_scattertext_explorer(corpus_phrases,
                                    category=category,
                                    category_name=category,
                                    not_category_name=not_category,
                                    minimum_term_frequency=3,
                                    pmi_threshold_coefficient=0,
                                    transform=dense_rank,
                                    # metadata=corpus.get_df()['speaker'],
                                    # term_scorer=RankDifference(),
                                    # term_scorer= st.ScaledFScorePresetsNeg1To1(beta=1, scaler_algo='normcdf'),
                                    term_scorer=st.LogOddsRatioInformativeDirichletPrior(priors_phrases, 10, 'class-size'),    
                                    width_in_pixels=1000)

open(output_dir+f'{filename}.html', 'w').write(html)





## Phrase scores using a different phrase-detection method textrank

In [ ]:


# Alternative phrase detection method 

%%time 

nlp = spacy.load("en_core_web_sm")


nlp.add_pipe("textrank", last=True)

df = train.assign(
    parse=lambda df: df[text_col_name].apply(nlp),
    # party=lambda df: df[group_col_name].apply({'democrat': 'Democratic', 'republican': 'Republican'}.get)
)
corpus = st.CorpusFromParsedDocuments(
    df,
    category_col=group_col_name,
    parsed_col='parse',
    feats_from_spacy_doc=st.PyTextRankPhrases()
).build(
).compact(
    AssociationCompactor(2000, use_non_text_features=True) # number of phrases displayed in the chart

)
# The phrases generated will be treated like non-textual features since their document scores will not correspond to word counts.


In [ ]:
term_ranks = np.argsort(np.argsort(-term_category_scores, axis=0), axis=0) + 1
metadata_descriptions = {
    term: '<br/>' + '<br/>'.join(
        '<b>%s</b> TextRank score rank: %s/%s' % (cat, term_ranks.loc[term, cat], corpus.get_num_metadata())
        for cat in corpus.get_categories())
    for term in corpus.get_metadata()
}
category_specific_prominence = term_category_scores.apply(
    # lambda r: r.Democratic if r.Democratic > r.Republican else -r.Republican,
    lambda r: r['Suicidal'] if r['Suicidal'] > r['Non-suicidal'] else -r['Non-suicidal'],
    axis=1
)

In [ ]:
html = produce_scattertext_explorer(
    corpus,
    category=category,
    not_category_name=not_category,
    minimum_term_frequency=2,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=dense_rank,
    # metadata=corpus.get_df()['speaker'],
    scores=category_specific_prominence,
    sort_by_dist=False,
    use_non_text_features=True,
    topic_model_term_lists={term: [term] for term in corpus.get_metadata()},
    topic_model_preview_size=0,
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True
)
open(output_dir+'explorer_collocations.html', 'w').write(html)

In [ ]:
html = produce_scattertext_explorer(
    corpus,
    category=category,
    not_category_name=not_category,
    minimum_term_frequency=2,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=dense_rank,
    use_non_text_features=True,
    # metadata=corpus.get_df()['speaker'],
    term_scorer=RankDifference(),
    sort_by_dist=False,
    topic_model_term_lists={term: [term] for term in corpus.get_metadata()},
    topic_model_preview_size=0, 
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True
)
open(output_dir+'explorer_collocations_2.html', 'w').write(html)

# How do the two types of post talk about suicide?

In this configuration of Scattertext, words are colored by their similarity to a query phrase.
This is done using spaCy-provided GloVe word vectors (trained on the Common Crawl corpus). The cosine distance between vectors is used, with mean vectors used for phrases.

The calculation of the most similar terms associated with each category is a simple heuristic. First, sets of terms closely associated with a category are found. Second, these terms are ranked based on their similarity to the query, and the top rank terms are displayed to the right of the scatterplot.

A term is considered associated if its p-value is less than 0.05. P-values are determined using Monroe et al. (2008)'s difference in the weighted log-odds-ratios with an uninformative Dirichlet prior. This is the only model-based method discussed in Monroe et al. that does not rely on a large, in-domain background corpus. Since we are scoring bigrams in addition to the unigrams scored by Monroe, the size of the corpus would have to be larger to have high enough bigram counts for proper penalization. This function relies the Dirichlet distribution's parameter alpha, a vector, which is uniformly set to 0.01.

In [ ]:
from scattertext import word_similarity_explorer
html = word_similarity_explorer(corpus_unigram,
                                 category=category,
                                 category_name=category,
                                 not_category_name=not_category,
                                 target_term='suicide',
                                 minimum_term_frequency=minimum_term_frequency,
                                 pmi_threshold_coefficient=0,
                                 width_in_pixels=1000,
                                
                                 # metadata=convention_df['speaker'],
                                 alpha=0.01,
                                 max_p_val=0.05,
                                 save_svg_button=True)
open(output_dir+"explorer_suicide.html", 'wb').write(html.encode('utf-8'))

# Maybe also compare to My other implementation using convokit code

# Explore with scores


In [ ]:

method_name = 'fscore'
if method_name == 'fscore':
    method = st.ScaledFScorePresets(beta=1, one_to_neg_one=True)


from sklearn.linear_model import Lasso
from scattertext import sparse_explorer
html = sparse_explorer(corpus_unigram,
                        category=category,
                        category_name=not_category,
                        not_category_name=not_category,
                        # scores = corpus.get_regression_coefs('democrat', Lasso(max_iter=10000)),
                           scores = method,
                        minimum_term_frequency=minimum_term_frequency,
                        pmi_threshold_coefficient=4,
                        width_in_pixels=1000,
                        # metadata=convention_df['speaker']
                      )




# semiotic square 

You need a neutral group (if comparing positive and negative opion sentiments, plot descriptions could be neutral)

https://github.com/JasonKessler/scattertext#creating-lexicalized-semiotic-squares

http://www.signosemio.com/greimas/semiotic-square.asp

In [ ]:
%%time
rdf = st.SampleCorpora.RottenTomatoes.get_data() #let's use rotten tomato which has neutral categories
rdf['category_name'] = rdf['category'].apply(lambda x: {'plot': 'Plot', 'rotten': 'Negative', 'fresh': 'Positive'}[x])
rdf['parse'] = rdf.text.apply(nlp)

phrase_corpus = (st.CorpusFromParsedDocuments(rdf, 
                                       category_col='category_name', 
                                       parsed_col='parse',
                                       feats_from_spacy_doc = st.PhraseMachinePhrases())
          .build())

In [ ]:
category = 'Positive'
not_category = 'Negative'
neutral_category_name = 'Plot'

In [ ]:
phrase_corpus_compact = corpus_pm.compact(st.CompactTerms(minimum_term_count=2))

semiotic_square = st.SemioticSquare(
    phrase_corpus_compact,
    category_a=category,
    category_b=not_category,
    neutral_categories=[neutral_category_name],
    scorer=st.RankDifference(), 
    labels = {'a_and_b': 'Posts',
              'not_a_and_not_b': 'Plot Descriptions',
              'a_and_not_b': f'{category}/Plot',
              'b_and_not_a': f'{not_category}/Plot',
             }
)

html = st.produce_semiotic_square_explorer(semiotic_square,
                                           category_name=category,
                                           not_category_name=not_category,
                                           x_label=f'{category}-{not_category}',
                                           y_label=f'Post-{neutral_category_name}',
                                           minimum_term_frequency=2,
                                           pmi_threshold_coefficient=0,
                                           neutral_category_name='Plot Description',
                                           # metadata=rdf['movie_name']
                                          )

open(output_dir+'semiotic_square_1.html', 'w').write(html)